# Daily report

Save the transactions from database files to csv files and generate a daily report.

In [1]:
import numpy as np
import pandas as pd
import sqlite3
from datetime import datetime

In [2]:
# connect the sqlite3 database
db = sqlite3.connect('products.db')
cursor = db.cursor()

# import the data from transactions record table of the database and convert the time into datetime
transactions = pd.read_sql_query(('SELECT * FROM t_record'), db)
transactions.time = pd.to_datetime(transactions.time)

In [3]:
display(transactions)

,transationid,time,productname,productid,amount,methodname
0,1,2017-12-13 11:38:37,Small Americano,14,1,Credit
1,1,2017-12-13 11:38:37,Kettle Chips Lighly Salted,72,1,Credit
2,1,2017-12-13 11:38:37,Small Latte,15,1,Credit
3,2,2017-12-13 11:38:49,Coca Cola 330ml,38,1,Debit
4,2,2017-12-13 11:38:49,Large Mocha,29,1,Debit
5,2,2017-12-13 11:38:49,Sea Salt Popcorn,65,1,Debit
6,2,2017-12-13 11:38:49,Medium Cappuccino,4,1,Debit
7,3,2017-12-13 11:39:03,Small Latte,15,1,Cash
8,3,2017-12-13 11:39:03,Coca Cola 330ml,38,1,Cash
9,3,2017-12-13 11:39:03,Panini Roast Beef Melt,44,1,Cash


In [4]:
# define a function that used to select a day
def select_time(dataframe, d, m, y):
    daily_transactions = dataframe[(dataframe.time.dt.day == d) & (dataframe.time.dt.month == m) 
                                   & (dataframe.time.dt.year == y)]
    
    return daily_transactions

In [5]:
# define a function that used to calculate the amounts of each method
def method_counts(data):
    cash = 0
    debit = 0
    credit = 0
    for i in data:
        if i == 'Cash':
            cash += 1
        if i == 'Debit':
            debit += 1
        if i == 'Credit':
            credit += 1
    
    return cash, debit, credit

In [6]:
# define a function that make a daily report
def report(dataframe, d, m, y):  
    
    dataframe = select_time(dataframe, d, m, y)
    
    mask = np.array(dataframe.productid)
    product_id, counts = np.unique(mask, return_counts = True)

    product_name = []
    product_price = []
    total_value = []
    total_cash = []
    total_debit = []
    total_credit = []

    for i in range(len(product_id)):
        cursor.execute('SELECT price FROM products WHERE productid = ?', (int(product_id[i]),))
        product_price.append(cursor.fetchone()[0] / 100)
        product_name.append(list(dataframe.loc[dataframe.productid == product_id[i]]['productname'])[0])  
        total_value.append(float("{0:.2f}".format(product_price[i] * counts[i])))
        method = list(dataframe.loc[dataframe.productid == product_id[i]]['methodname'])
        method = method_counts(method)
        total_cash.append(method[0])
        total_debit.append(method[1])
        total_credit.append(method[2])

    column_name = ['product_name', 'product_id','product_price', 'sales_counts','sales_values', 
                   'Paid by Cash amounts', 'Paid by Debit amounts', 'Paid by Credit amounts']

    w_values = np.column_stack((product_name, product_id, product_price, counts, total_value, 
                                total_cash, total_debit, total_credit))

    report = pd.DataFrame(columns = column_name,data = w_values)
    
    return report

In [7]:
# for example, report the transactions daily report on 2017/12/13
daily_20171213 = report(transactions, 13, 12, 2017)

In [8]:
display(daily_20171213)

,product_name,product_id,product_price,sales_counts,sales_values,Paid by Cash amounts,Paid by Debit amounts,Paid by Credit amounts
0,Medium Latte,3,2.0,1,2.0,0,1,0
1,Medium Cappuccino,4,2.0,5,10.0,1,3,1
2,Medium Mocha,5,2.2,2,4.4,2,0,0
3,Medium Hot Chocolate,6,2.1,1,2.1,1,0,0
4,Small Americano,14,1.6,1,1.6,0,0,1
5,Small Latte,15,1.8,10,18.0,5,1,4
6,Small Cappuccino,16,1.8,2,3.6,1,0,1
7,Large Latte,27,2.25,1,2.25,1,0,0
8,Large Mocha,29,2.6,2,5.2,0,1,1
9,Coca Cola 330ml,38,1.0,11,11.0,5,4,2


In [9]:
# define a function that used to report a target product during the period
def product_report(dataframe, target_product_id, start_datetime, end_datetime):
    start_datetime = pd.to_datetime(start_datetime)
    end_datetime = pd.to_datetime(end_datetime)
    
    # select the target product dataframe
    product_data = dataframe.loc[dataframe.productid == target_product_id]
    
    # select the period dataframe
    product_data = product_data.loc[(product_data.time > start_datetime) & (product_data.time < end_datetime)]
    
    # insert the date used for group by
    product_data.insert(6, 'date', product_data.time.dt.date, allow_duplicates=False)
    
    # insert the price
    cursor.execute('SELECT price FROM products WHERE productid = ?', (target_product_id,))
    target_price = cursor.fetchone()[0] / 100
    product_data.insert(7, 'price', target_price, allow_duplicates=False)
    
    # group by
    product_report = product_data.groupby(['date','productname'])['amount', 'price'].sum()
    
    product_report = product_report.rename(index = str, 
                                           columns = {'amount': 'total_volume', 'price': 'total_value'})
    
    product_report.reset_index(level=(0,1), inplace=True)

    return product_report

In [10]:
# for example, select productid == 38 for the following time period
start_time = '13-12-2017 11:41:00'
end_time = '13-12-2017 11:46:00'
target_product_id = 38

report_38 = product_report(transactions, target_product_id, start_time, end_time)

In [11]:
display(report_38)

,date,productname,total_volume,total_value
0,2017-12-13,Coca Cola 330ml,4,4.0


In [12]:
# export the data into csv files
transactions.to_csv('till_tansactions.csv', encoding = 'utf-8', index = False)

daily_20171213.to_csv('Daily_report_20171213.csv', encoding = 'utf-8', index = False)

report_38.to_csv('Product_report_38.csv', encoding = 'utf-8', index = False)